In [1]:
!wget https://github.com/narain1/dotfiles/releases/download/v1.0.5/Test.zip -q

In [2]:
!unzip -q Test.zip

In [3]:
import os
import pandas as pd
import json
import warnings
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

warnings.filterwarnings('ignore')

In [4]:
xl_fs = list(map(lambda x: os.path.join('Test', x), os.listdir('Test')))

In [5]:
def filter_nulls(o):
    d1 = dict()
    for k, v in d.items():
        d2 = dict()
        for k1, v1 in v.items():
            if v1 != None:
                d2[k1] = v1
        d1[k] = d2
    return d1

In [6]:
os.makedirs('json_files', exist_ok=True)
for f in xl_fs:
    sheet_names = pd.ExcelFile(f).sheet_names
    for s in sheet_names:
        df = pd.read_excel(f, sheet_name=s).dropna(how='all')
        d = json.loads(df.to_json())
        d = filter_nulls(d)
        with open(f'json_files/{os.path.basename(f)}_{s}.json', 'w') as f_write:
            json.dump(d, f_write, indent=4)

In [7]:
json_files = xl_fs = list(map(lambda x: os.path.join('json_files', x), os.listdir('json_files')))
len(json_files)

512

In [8]:
acc = []
for f in json_files:
    acc1 = []
    with open(f) as f_read:
        o = json.load(f_read)
    assert o['SIEMENS']['9'] in ['Menge', 'Quantity']
    assert len(o) == 11
    headers = [o[k].get('9', '') for k in o]
    keys = o[list(o.keys())[0]].keys()
    for k in keys:
        if int(k) > 9:
            acc1.append({i:o[j].get(k, '') for i,j in zip(headers, o.keys())})
    acc.append(acc1)

In [9]:
def stringify_sheet(a):
    acc = []
    for b in a:
        for k,v in b.items():
            if '' in [k,v]: continue
            acc.append(f"{str(k).replace(' ', '-')}__{str(v).replace(' ', '-')}")
    return ' '.join(acc)

In [10]:
tfidf = TfidfVectorizer()
query_matrix = tfidf.fit_transform(list(map(stringify_sheet, acc)))

In [11]:
query_matrix.shape

(512, 5868)

In [23]:
rank_values = query_matrix @ tfidf.transform([stringify_sheet(acc[0])]).T
rank_values = np.array(rank_values.todense()).squeeze()
# rank_values = rank_values

In [41]:
top_5 = np.argsort(rank_values)[-5:]

In [42]:
acc[top_5[0]]

[{'Menge': 1,
  'Bestell-Nr.': '3AX4206',
  'Kurztext': 'klein-Positionsschalter',
  '': '',
  'BMKZ': '-S80',
  'Ort': '+.B',
  'Bem.': '',
  'Whrg.': 'EUR',
  'L-Preis Gerät\n(einzel)': 2,
  'L-Preis E&V\n(einzel)': 81.6165536,
  'L-Preis Summe\n(gesamt)': 83.6165536},
 {'Menge': 1,
  'Bestell-Nr.': '3RH2440-1BF40',
  'Kurztext': 'HILFSSCHUETZ VERKL,4S,DC110V',
  '': '',
  'BMKZ': '-K1',
  'Ort': '+.B',
  'Bem.': 'CONT.RELAY LATCHED,4NO,DC110V',
  'Whrg.': 'EUR',
  'L-Preis Gerät\n(einzel)': 78.54,
  'L-Preis E&V\n(einzel)': 155.456384,
  'L-Preis Summe\n(gesamt)': 233.996384},
 {'Menge': 1,
  'Bestell-Nr.': '3RT2916-1DG00',
  'Kurztext': 'ENTSTOERDIODE DC12-250V (F. S00)',
  '': '',
  'BMKZ': '-K1',
  'Ort': '+.B',
  'Bem.': 'SIRIUS NG CONTACTORS ACCESSORY',
  'Whrg.': 'EUR',
  'L-Preis Gerät\n(einzel)': 7.43,
  'L-Preis E&V\n(einzel)': 0,
  'L-Preis Summe\n(gesamt)': 7.43},
 {'Menge': 1,
  'Bestell-Nr.': '3SU1000-0AB20-0AA0',
  'Kurztext': 'DRUCKTASTER, ROT',
  '': '',
  'BMKZ': '-

In [40]:
top_5

array([151, 487,  91, 268, 196, 444, 190, 468, 342, 388, 350, 144, 312,
       445, 500, 480, 240, 153, 116, 218, 474, 282, 192, 502, 220, 258,
       506, 407, 149, 292,  89, 239, 140, 274, 290,  72,  51, 458, 461,
       377, 464, 408, 247, 352, 121, 198,   3, 194, 302, 254, 317, 493,
       167, 343, 504, 313, 234, 230, 354, 181, 365, 346, 509, 404,  33,
       508,  88, 393, 358, 199, 131, 104, 215, 148, 242, 310, 384, 494,
       224,   6,  36, 156, 206, 439, 138, 492, 443, 277, 486,  87,  20,
       102, 321, 347, 150, 460, 415, 398, 436, 334, 134, 477, 256, 366,
       505,  70,  86,  53,  79, 376, 471, 428, 111, 390,  44,  38, 241,
        90, 332, 434,   2, 289, 449, 205, 298, 467, 291, 184,  68,  29,
        55, 470, 276,   4, 371, 296, 314,  93, 231, 147,  76, 106, 397,
       204, 187, 341,  26, 490, 214, 392, 163, 269, 511,  43, 381, 232,
       109, 133, 223, 356,  48, 433,  50,  95, 278, 279, 395, 161, 425,
       237, 119, 336, 462, 263, 172,  74, 331, 359, 202, 319, 35